https://www.kaggle.com/code/rkuo2000/chromadb-langchain-gpt4all

https://python.langchain.com/docs/integrations/llms/gpt4all

https://github.com/nomic-ai/gpt4all/tree/main/gpt4all-bindings/python

https://www.kaggle.com/code/gpreda/test-gpt4all-on-kaggle

In [1]:
import time
start = time.time()

In [2]:
! nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-104c8356-6afb-2580-0024-8f3db933580a)
GPU 1: Tesla T4 (UUID: GPU-7b6c8258-d60e-95a8-0afc-bbb7514d4d2c)


In [3]:
%%time

! pip install -qq -U langchain tiktoken pypdf chromadb faiss-gpu
! pip install -qq -U transformers InstructorEmbedding sentence_transformers
! pip install -qq -U bitsandbytes bitsandbytes-cuda117
! pip install -qq -U gpt4all

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.3 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.12.1 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2023.12.1 which is incompatible.
dask-cuda 23.8.0 requires dask==2023.7.1, b

In [4]:
#!wget https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin models

In [5]:
import warnings
warnings.filterwarnings("ignore")

import os
import textwrap
import shutil
import pandas as pd
import glob

import langchain
from langchain.llms import HuggingFacePipeline

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline

#quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)

#os.environ['TRANSFORMERS_CACHE'] = 'cache/'

print(langchain.__version__)

0.1.4


In [6]:
### Multi-document retriever
from langchain.vectorstores import Chroma, FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA, VectorDBQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
#from gpt4all import GPT4All
from langchain.llms import GPT4All
from langchain.prompts import PromptTemplate


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [7]:
!wget https://gpt4all.io/models/gguf/mistral-7b-openorca.Q4_0.gguf

--2024-01-30 01:57:59--  https://gpt4all.io/models/gguf/mistral-7b-openorca.Q4_0.gguf
Resolving gpt4all.io (gpt4all.io)... 172.67.71.169, 104.26.1.159, 104.26.0.159, ...
Connecting to gpt4all.io (gpt4all.io)|172.67.71.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4108927744 (3.8G)
Saving to: 'mistral-7b-openorca.Q4_0.gguf'

mistral-7b-openorca 100%[===================>]   3.83G  7.67MB/s    in 7m 53s  

2024-01-30 02:05:53 (8.28 MB/s) - 'mistral-7b-openorca.Q4_0.gguf' saved [4108927744/4108927744]



In [8]:

llm = GPT4All(model="/kaggle/working/mistral-7b-openorca.Q4_0.gguf")

In [9]:
#print(llm.generate("The capital of France is ", max_tokens=1028))

In [10]:
llm

GPT4All(model='/kaggle/working/mistral-7b-openorca.Q4_0.gguf', client=<gpt4all.gpt4all.GPT4All object at 0x7dae2028fdf0>)

In [11]:
shutil.copytree('/kaggle/input/mkvc-6-finalmix/kaggle/working/mk-vectordb-chroma-six', '/kaggle/working/mk-vectordb-chroma')

'/kaggle/working/mk-vectordb-chroma'

In [12]:
%%time

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="intfloat/multilingual-e5-base", model_kwargs={"device": "cuda"})

vectordb = Chroma(persist_directory='mk-vectordb-chroma',embedding_function=instructor_embeddings,)
#vectordb.persist()

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

TypeError: INSTRUCTOR._load_sbert_model() got an unexpected keyword argument 'token'

In [13]:
%%time
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
#retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 6, "lambda_mult": 0.25})

qa_chain = RetrievalQA.from_chain_type(llm=llm, 
                                       chain_type="stuff", 
                                       retriever=retriever, 
                                       return_source_documents=True,
                                       verbose=False)

NameError: name 'vectordb' is not defined

In [14]:
qa_chain

NameError: name 'qa_chain' is not defined

In [ ]:
def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
def llm_ans(query):
    llm_response = qa_chain(query)
    ans = process_llm_response(llm_response)
    return ans

In [ ]:
#model

In [ ]:
dataset = pd.read_csv('/kaggle/input/mkri-qa-3/dataset.csv',sep=';')
dataset

In [ ]:
%%time
for x,y in zip(dataset.no,dataset.qa_comb):
    llm_time_start = time.time()
    print("Nomor: ",x)
    print(y)
    llm_ans(y)
    llm_time_end = time.time()
    print("Time: ",llm_time_end - llm_time_start)
    print("==============================================")

In [ ]:
end = time.time()
print(end - start)